# Phase 2: Break It

In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
TRAIN = '../data/interim/train.csv'

In [36]:
input_data = []
label = []
with open(TRAIN, mode='r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        input_data.append(lines[0])
        label.append(int(lines[1]))
        
input_data = input_data[0:100]
label = label[0:100]

In [63]:
input_data[0:2]

['Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!',
 'Shame Shame This tape can hardly be understood and it was listed for sale as "very good".  It\'s VERY BAD.']

In [37]:
print(f'Sentence 1:\n{input_data[0]}\nLabel: {label[0]}')
type(input_data)

Sentence 1:
Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!
Label: 1


list

In [38]:
#conda install checklist
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

In [39]:
#python3 -m spacy download en_core_web_sm
#conda install spacy
import spacy
nlp = spacy.load('en_core_web_sm')

# POS tag data

https://spacy.io/usage/processing-pipelines

When you call nlp on a text, spaCy will tokenize it and then call each component on the Doc, in order. It then returns the processed Doc that you can work with.

In [60]:
pdata = list(nlp.pipe(input_data))
#for doc in nlp.pipe(input_data):
    # Do something with the doc here
#    print([(ent.text, ent.label_) for ent in doc.ents])

# Remove End Punctuation

In [41]:
pdata[0], Perturb.strip_punctuation(pdata[0])

(Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!,
 'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago')

In [69]:
ret = Perturb.perturb(pdata, Perturb.punctuation)
ret.data[0:2]

[['Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!',
  'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago',
  'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago.'],
 ['Shame Shame This tape can hardly be understood and it was listed for sale as "very good".  It\'s VERY BAD.',
  'Shame Shame This tape can hardly be understood and it was listed for sale as "very good".  It\'s VERY BAD']]

# Add Typos

In [53]:
input_data[0], Perturb.add_typos(input_data[0])

('Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!',
 'Gotta listen to this! So creative!  Love ihs music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!')

In [73]:
ret = Perturb.perturb(input_data, Perturb.add_typos, nsamples=1)
ret.data

[['Greatest Songs, Worst Rendition These may be, and are, some of America\'s Greatest Old-time Gospel songs, but the way they are presented is an abomination!  Gospel, especially Old-Time Gospel is music of the soul, sung from the heart.  Neither heart nor soul can be found in these arrangements.  I would compare the redition of these classics to fast food, but that wouldn\'t really be fair, since fast food has some nutritional value. I found this music to be as plastic as the CD it was burned on, with no "nutritional value" to the heart and soul whatsoever.\n\nIt doesn\'t even deserve one star.',
  'Greatest Songs, Worst eRndition These may be, and are, some of America\'s Greatest Old-time Gospel songs, but the way they are presented is an abomination!  Gospel, especially Old-Time Gospel is music of the soul, sung from the heart.  Neither heart nor soul can be found in these arrangements.  I would compare the redition of these classics to fast food, but that wouldn\'t really be fair, 

# Negation

In [80]:
ret = Perturb.perturb(pdata, Perturb.remove_negation)
ret.data[0]

["Buy the CD.  Do not buy the MP3. Buy the CD.  Do not buy the MP3 album.  Download is no longer available.  But you don't find that out until after you have purchased it.",
 'Buy the CD.   buy the MP3. Buy the CD.   buy the MP3 album.  Download is longer available.  But you find that out until after you have purchased it.']